In [ ]:
from matplotlib import pyplot as plt
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

plt.style.use('Solarize_Light2')

- **Load PCEPILFE data**

In [18]:
PCEPILFE_df = pd.read_csv('../data/PCEPILFE.csv')

PCEPILFE_df['DATE'] = pd.to_datetime(PCEPILFE_df['DATE'])
PCEPILFE_df.set_index('DATE', inplace=True)

PCEPILFE_df['^ %'] = PCEPILFE_df['PCEPILFE'].pct_change()

PCEPILFE_df.tail()

,PCEPILFE,^ %
DATE,,
2025-05-01,125.790,0.002295
2025-06-01,126.121,0.002631
2025-07-01,126.426,0.002418
2025-08-01,126.707,0.002223
2025-09-01,126.955,0.001957


- **Average change**

In [19]:
print(f"Average change is {PCEPILFE_df['^ %'].mean() * 100}%")

Average change is 0.26339964002544225%


- **Overview on change**

In [20]:
positive_filter = PCEPILFE_df['^ %'] > 0
negative_filter = PCEPILFE_df['^ %'] < 0
zero_filter = PCEPILFE_df['^ %'] == 0

# count
total_items = len(PCEPILFE_df)

# average change
avg_change = [
    PCEPILFE_df[positive_filter]['^ %'].mean(),
    PCEPILFE_df[negative_filter]['^ %'].mean(),
    0,
    ""
]

# frequency
frequency = [
    PCEPILFE_df[positive_filter]['^ %'].count(),
    PCEPILFE_df[negative_filter]['^ %'].count(),
    PCEPILFE_df[zero_filter]['^ %'].count()
]
frequency.append(frequency[0]/frequency[1])

# frequency pct
frequency_pct = [100 * frequency[i]/total_items for i in range(3)]
frequency_pct.append("")

# Prob adj
prob_adj = [frequency_pct[i] * avg_change[i] for i in range(3)]
prob_adj.append("")

pd.DataFrame({
    "%": [x * 100 for x in avg_change],
    "Frequency": frequency,
    "Frequency %": frequency_pct,
    "Prob Adjust % Change": prob_adj,
}, index=["Av Pos", "Av Neg", "Zero", "Ratio P/N"])

,%,Frequency,Frequency %,Prob Adjust % Change
Av Pos,0.271193,783.000000,97.752809,0.265099
Av Neg,-0.095553,17.000000,2.122347,-0.002028
Zero,0,0.000000,0.0,0.0
Ratio P/N,,46.058824,,


- **Stats**

In [21]:
from scipy.stats import describe

# Display the results

stats = describe(PCEPILFE_df['^ %'].dropna().tolist())
pd.DataFrame(
    {
        'value': [
            str(stats.nobs),
            stats.minmax[0] * 100,
            stats.minmax[1] * 100,
            stats.mean * 100,
            PCEPILFE_df['^ %'].median() * 100,
            PCEPILFE_df['^ %'].mode(dropna=True)[0] * 100,
            stats.variance,
            stats.skewness,
            stats.kurtosis
        ]
    },
    index=['nobs', 'Min %', 'Max %', 'Mean %', "Median %", "Mode %", 'Variance', 'Skewness', 'Kurtosis'],
)

,value
nobs,800
Min %,-0.568234
Max %,1.025997
Mean %,0.2634
Median %,0.211787
Mode %,-0.568234
Variance,0.000004
Skewness,0.963523
Kurtosis,1.554929


- **Data preview**

In [22]:
# Define bins
bins = [0.005 * i for i in range(-1, 3)]
bins.append(stats.minmax[1])
bins.insert(0, stats.minmax[0])
bin_labels = [f"{round(bins[i] * 100, 2)}% to {round(bins[i+1] * 100, 2)}%" for i in range(len(bins) - 1)]
bin_labels[0] = f"Less than {bins[1] * 100}%"
bin_labels[-1] = f"Greater than {bins[-2] * 100}%"

# Assign data to bins
binned = pd.cut(PCEPILFE_df['^ %'], bins=bins, labels=bin_labels, include_lowest=True)

# Calculate frequency, probability, and cumulative probability
frequency = binned.value_counts().sort_index()
probability = 100 * frequency / frequency.sum()
cumulative_probability = probability.cumsum()

occurrence_frequencies = pd.DataFrame({
    'Frequency': frequency.values,
    'Probability %': probability.values,
    'Cumulative Probability %': cumulative_probability.values
}, index=bin_labels)

occurrence_frequencies

,Frequency,Probability %,Cumulative Probability %
Less than -0.5%,1,0.125,0.125
-0.5% to 0.0%,16,2.000,2.125
0.0% to 0.5%,682,85.250,87.375
0.5% to 1.0%,100,12.500,99.875
Greater than 1.0%,1,0.125,100.000


In [23]:
fig = px.bar(occurrence_frequencies, x=occurrence_frequencies.index, y='Probability %')

fig = px.bar(
    occurrence_frequencies,
    x=occurrence_frequencies.index,
    y='Probability %',
)

fig.update_layout(
    title=dict(text='PCEPILFE % change'),
    width=800,
    height=600,
    yaxis=dict(title="Frequency"),
    hovermode="x unified",
    template="plotly_dark"
)

fig.show()

- **PCEPILFE % change**

In [24]:
fig = go.Figure()

# Reference line
fig.add_trace(go.Scatter(
    x=PCEPILFE_df.index,
    y=[100 * stats.mean] * (stats.nobs + 1),
    name='Average % change',
    line=dict(
        width=2,
        color="#F2C14E",   # muted gold
        dash="dash"
    ),
    hoverinfo="skip",
    # showlegend=False,
))

# Bars
fig.add_trace(go.Bar(
    x=PCEPILFE_df.index,
    y=PCEPILFE_df['^ %'] * 100,
    name='PCEPILFE',
    marker=dict(
        color="#4C78A8",  # professional blue
        line=dict(color="#4C78A8", width=0)
    ),
    width=0.5,
))

fig.update_layout(
    title=dict(
        text='PCEPILFE % change',
        x=0.02,
        font=dict(size=20)
    ),
    plot_bgcolor="#0E1117",
    paper_bgcolor="#0E1117",
    font=dict(color="#E6E6E6", size=14),
    width=1400,
    height=700,
    yaxis=dict(
        title="%",
        exponentformat="none",
        showgrid=False,
        zeroline=False
    ),
    xaxis=dict(
        showgrid=False,
        zeroline=False
    ),
    hovermode="x unified",
)

fig.show()


- **WPSFD49207 data plot**

In [25]:
fig = px.line(
    PCEPILFE_df,
    x=PCEPILFE_df.index,
    y="PCEPILFE",
    markers=True
)

fig.update_layout(
    title=dict(text='Personal Consumption Expenditures Excluding Food and Energy (Chain-Type Price Index)'),
    width=1000,
    height=600,
    yaxis=dict(title="PCEPILFE"),
    hovermode="x unified",
    template="plotly_dark"
)

fig.show()